In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):

    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])

    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성.
    new_feature_name_df = get_new_feature_name_df(feature_name_df)

    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()

    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt',sep='\s+', names=feature_name)

    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt',sep='\s+',header=None,names=['action'])

    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

In [5]:
# 결정 트리에서 사용한 get_human_dataset( )을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7352, 561), (2947, 561), (7352, 1), (2947, 1))

In [7]:
# Scale train/ test predictors based on training data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
##CNN, LSTM을 위해 3차원 데이터로 shape 변형한다.

# Reformat the training data into sequence samples
# Source: https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)



In [9]:
# Merge train and test X/y data to apply sequence transformation function
y_train_array = np.array(y_train)
train_set = np.c_[X_train, y_train_array]

y_test_array = np.array(y_test)
test_set = np.c_[X_test, y_test_array]


# Apply sequence transformation using time step of 25 for both train and test data
# All sensing modalities are recorded at a sampling rate of 50 Hz, which is considered sufficient for capturing human activity.

X_train, y_train = split_sequences(train_set, 25)
print(X_train.shape, y_train.shape)

X_test, y_test = split_sequences(test_set, 25)
print(X_test.shape, y_test.shape)

(7328, 25, 561) (7328,)
(2923, 25, 561) (2923,)


In [10]:
print(X_train.shape, y_train.shape)

(7328, 25, 561) (7328,)


In [11]:
# Convert output variables to categorical for CNN
y_train = to_categorical(y_train)
print(y_train.shape)

y_test = to_categorical(y_test)
print(y_test.shape)

(7328, 7)
(2923, 7)


In [12]:
# Produce variables for CNN input/output shape
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
print(n_timesteps, n_features, n_outputs)

25 561 7


##CNN

In [15]:
## set number of epochs and batch size
batch_size = 64
validation_split = 0.2
train_epochs = 20

In [13]:


#모델 생성
model = Sequential()

#컨벌루션(합성곱)레이어 추가
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5)) #과적합방지
model.add(MaxPooling1D(pool_size=2))

#flatten
model.add(Flatten())

#denselayer 추가
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 22, 64)            143680    
                                                                 
 dropout (Dropout)           (None, 22, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 11, 64)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 704)               0         
                                                                 
 dense (Dense)               (None, 100)               70500     
                                                                 
 dense_1 (Dense)             (None, 7)                 707       
                                                        

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
#train model
# fit network

history = model.fit(X_train, y_train, epochs=train_epochs, batch_size=batch_size, verbose=True,  validation_split=validation_split, shuffle=True)

Epoch 1/20
92/92 [==============================] - 6s 45ms/step - loss: 0.7295 - accuracy: 0.7704 - val_loss: 0.2355 - val_accuracy: 0.9291
Epoch 2/20
92/92 [==============================] - 3s 35ms/step - loss: 0.2057 - accuracy: 0.9280 - val_loss: 0.2750 - val_accuracy: 0.9154
Epoch 3/20
92/92 [==============================] - 5s 53ms/step - loss: 0.1124 - accuracy: 0.9616 - val_loss: 0.1461 - val_accuracy: 0.9482
Epoch 4/20
92/92 [==============================] - 6s 67ms/step - loss: 0.0894 - accuracy: 0.9686 - val_loss: 0.1915 - val_accuracy: 0.9407
Epoch 5/20
92/92 [==============================] - 3s 35ms/step - loss: 0.0775 - accuracy: 0.9727 - val_loss: 0.1135 - val_accuracy: 0.9632
Epoch 6/20
92/92 [==============================] - 3s 35ms/step - loss: 0.0555 - accuracy: 0.9819 - val_loss: 0.0908 - val_accuracy: 0.9652
Epoch 7/20
92/92 [==============================] - 4s 41ms/step - loss: 0.0507 - accuracy: 0.9819 - val_loss: 0.1155 - val_accuracy: 0.9557
Epoch 8/20
92

In [17]:
model.evaluate(X_test, y_test)

92/92 [==============================] - 1s 8ms/step - loss: 0.1714 - accuracy: 0.9644


[0.17141924798488617, 0.9644201397895813]

##CNN-LSTM

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense

#n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]


#모델 생성
model = Sequential()

#컨벌루션(합성곱)레이어 추가
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5)) #과적합방지
model.add(MaxPooling1D(pool_size=2))

# LSTM 레이어 추가
model.add(LSTM(64))

#flatten
model.add(Flatten())

#denselayer 추가
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 22, 64)            143680    
                                                                 
 dropout_1 (Dropout)         (None, 22, 64)            0         
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 11, 64)            0         
 g1D)                                                            
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 100)               6500      
                                                      

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
#train model
# fit network

history = model.fit(X_train, y_train, epochs=train_epochs, batch_size=batch_size, verbose=True,  validation_split=validation_split, shuffle=True)

Epoch 1/20
92/92 [==============================] - 8s 56ms/step - loss: 0.5559 - accuracy: 0.8019 - val_loss: 0.2821 - val_accuracy: 0.9188
Epoch 2/20
92/92 [==============================] - 5s 51ms/step - loss: 0.1540 - accuracy: 0.9473 - val_loss: 0.2679 - val_accuracy: 0.9209
Epoch 3/20
92/92 [==============================] - 6s 62ms/step - loss: 0.0815 - accuracy: 0.9739 - val_loss: 0.1029 - val_accuracy: 0.9679
Epoch 4/20
92/92 [==============================] - 4s 44ms/step - loss: 0.0698 - accuracy: 0.9761 - val_loss: 0.1423 - val_accuracy: 0.9482
Epoch 5/20
92/92 [==============================] - 4s 47ms/step - loss: 0.0496 - accuracy: 0.9836 - val_loss: 0.0836 - val_accuracy: 0.9768
Epoch 6/20
92/92 [==============================] - 7s 75ms/step - loss: 0.0499 - accuracy: 0.9826 - val_loss: 0.1102 - val_accuracy: 0.9638
Epoch 7/20
92/92 [==============================] - 5s 56ms/step - loss: 0.0356 - accuracy: 0.9886 - val_loss: 0.1293 - val_accuracy: 0.9652
Epoch 8/20
92

In [21]:
model.evaluate(X_test, y_test)

92/92 [==============================] - 1s 10ms/step - loss: 0.1331 - accuracy: 0.9620


[0.13310837745666504, 0.9620253443717957]